# RDFS NN

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.layers import Input, Dense, Dropout, Bidirectional, GRU, RepeatVector, TimeDistributed
from keras.models import Model

2023-03-22 18:48:01.429882: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 18:48:02.362750: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-22 18:48:02.362795: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-22 18:48:05.609745: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
import tensorflow as tf

if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:
    print("Please install GPU version of TF")


Please install GPU version of TF


2023-03-22 18:48:08.605749: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 18:48:08.671791: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-22 18:48:08.671869: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (archangel): /proc/driver/nvidia/version does not exist


In [3]:
col_names=["subject", "predicate", "object"]
df = pd.read_csv("./data/jupyter/all_lubm.nt", delimiter=' ', names=col_names)
df['object'] = df['object'].apply(lambda x: x.rstrip('.'))
print("ready")

ready


In [4]:
df.head()

,subject,predicate,object
0,<http://www.Department1.University0.edu/Underg...,<http://swat.cse.lehigh.edu/onto/univ-bench.ow...,<http://www.Department1.University0.edu>
1,<http://www.Department1.University0.edu/Underg...,<http://swat.cse.lehigh.edu/onto/univ-bench.ow...,<http://www.Department1.University0.edu/Course42>
2,<http://www.Department1.University0.edu/Underg...,<http://swat.cse.lehigh.edu/onto/univ-bench.ow...,<http://www.Department1.University0.edu/Course25>
3,<http://www.Department1.University0.edu/Underg...,<http://swat.cse.lehigh.edu/onto/univ-bench.ow...,xxx-xxx-xxxx
4,<http://www.Department1.University0.edu/Underg...,<http://swat.cse.lehigh.edu/onto/univ-bench.ow...,UndergraduateStudent98


In [5]:
subjects = df["subject"].unique()
predicates = df["predicate"].unique()
objects = df["object"].unique()

In [6]:
df_encoded = df.copy()

In [7]:
encoded_identifiers = {}
triples = zip(df["subject"], df["predicate"], df["object"])
    

subject_to_id = {subject: i for i, subject in enumerate(subjects)}
predicate_to_id = {predicate: i for i, predicate in enumerate(predicates)}
object_to_id = {object: i for i, object in enumerate(objects)}

In [8]:
s_l = subjects.tolist()
p_l = predicates.tolist()
o_l = objects.tolist()

s_l.extend(p_l)
s_l.extend(o_l)

s_l = list(enumerate(set(s_l)))

encoding_dict = {id:encoded_id for (encoded_id,id) in s_l}
encoding_dict


{'<http://www.Department5.University0.edu/FullProfessor7/Publication2>': 0,
 '<http://www.Department9.University0.edu/AssistantProfessor8/Publication4>': 1,
 '<http://www.Department4.University0.edu/FullProfessor6/Publication5>': 2,
 'UndergraduateStudent43@Department10.University0.edu': 3,
 '<http://www.Department3.University0.edu/AssociateProfessor7/Publication17>': 4,
 'GraduateStudent80@Department0.University0.edu': 5,
 '<http://www.Department7.University0.edu/UndergraduateStudent192>': 6,
 '<http://www.Department5.University0.edu/UndergraduateStudent107>': 7,
 'UndergraduateStudent235': 8,
 '<http://www.University848.edu>': 9,
 '<http://www.Department1.University0.edu/FullProfessor1/Publication17>': 10,
 'UndergraduateStudent253@Department2.University0.edu': 11,
 '<http://www.Department3.University0.edu/UndergraduateStudent212>': 12,
 'UndergraduateStudent186@Department10.University0.edu': 13,
 '<http://www.Department13.University0.edu/Course25>': 14,
 '<http://www.Department6.Uni

In [9]:
encoding_dict['GraduateCourse23']
df_encoded["subject"] = df_encoded["subject"].apply(lambda x: encoding_dict[x])
df_encoded["predicae"] = df_encoded["predicate"].apply(lambda x: encoding_dict[x])
df_encoded["object"] = df_encoded["object"].apply(lambda x: encoding_dict[x])

In [10]:
df_encoded['subject']

0         14245
1         14245
2         14245
3         14245
4         14245
          ...  
100568    15471
100569    15471
100570    13099
100571    13099
100572    13099
Name: subject, Length: 100573, dtype: int64

In [13]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from sklearn.model_selection import train_test_split

def create_dataframe_translation_model_LSTM(df):
    input_texts = df['subject'].str.cat(df['predicate'], sep=' ')
    target_texts = df['object']

    # Create input and target character index mappings
    input_characters = sorted(set(' '.join(input_texts)))
    target_characters = sorted(set(' '.join(target_texts)))
    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)

    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

    # Encode input and target sequences as one-hot vectors
    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])

    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
    decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.
        for t, char in enumerate(target_text):
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.

    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(encoder_input_data, decoder_input_data, test_size=0.2, random_state=42)

    # Define and compile the NMT model
    latent_dim = 256
    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None, num_decoder_tokens))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the NMT model
    batch_size = 16
    epochs = 50
    history = model.fit([X_train, y_train], y_train, batch_size=batch_size, epochs=epochs, validation_data=([X_val, y_val], y_val))

    return model

In [14]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Dropout, GRU, Bidirectional, RepeatVector, TimeDistributed
from sklearn.model_selection import train_test_split

def create_dataframe_translation_model_GRU(df):
    # Create input and target text sequences
    input_texts = df.apply(lambda row: row['subject'] + ' ' + row['predicate'], axis=1)
    target_texts = df['object']

    # Create input and target character index mappings
    input_characters = sorted(set(' '.join(input_texts)))
    target_characters = sorted(set(' '.join(target_texts)))
    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)

    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

    # Encode input and target sequences as one-hot vectors
    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])

    encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
    decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.
        for t, char in enumerate(target_text):
            decoder_input_data[i, t, target_token_index[char]] = 1.
            if t > 0:
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.

    # Split data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(encoder_input_data, decoder_input_data, test_size=0.2, random_state=42)

    # Define and compile the NMT model with GRU layers
    latent_dim = 256
    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = Bidirectional(GRU(latent_dim, name="gru_sequence_encoder"), name='bidirectional')
    encoder_outputs = encoder(encoder_inputs)

    decoder_inputs = Input(shape=(None, num_decoder_tokens))
    decoder = GRU(latent_dim * 2, return_sequences=True, name='sequence_decoder')
    decoder_outputs = decoder(decoder_inputs, initial_state=encoder_outputs)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the NMT model
    batch_size = 16
    epochs = 1
    history = model.fit([X_train, y_train[:, :-1, :]], y_train[:, 1:, :], batch_size=batch_size, epochs=epochs, validation_data=([X_val, y_val[:, :-1, :]], y_val[:, 1:, :]))

    return model

In [ ]:
model = create_dataframe_translation_model_GRU(df)

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Define the input and output dimensions of the model
input_dim = 100
output_dim = 50

latent_dim=256
batch_size=64
epochs=1

# Define the number of hidden units in the model
hidden_units = 256

In [ ]:
# Initialize the GraphTranslationModel object
# model = GraphTranslationModel(df_train, latent_dim=latent_dim, batch_size=batch_size, epochs=epochs)

In [ ]:
translations = model.translate(df_test)